<a href="https://colab.research.google.com/github/danielriosgarza/microbesGYM/blob/main/notebooks/kinetic_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Building and simulating a kinetic microbiome model**

### **Install the package and dependencies**

In [ ]:
!git clone https://github.com/danielriosgarza/microbesGYM.git

In [ ]:
%cd microbesGYM
!uv pip install .[all]

In [3]:
from google.colab import output
output.enable_custom_widget_manager()

### **Kinetic model environment**

The `kinetic model` environment allows us to simulate microbial communities and their environments using ordinary differential equations.

To illustrate how it works, we build a simplified examplt with *E. coli* subpopulations.

In this example, there are three subpopulations: two active and one inactive.

- **Subpopulation A** consumes glucose and produces acetate (overflow).

- **Subpopulation B** consumes acetate and produces CO2. Both subpopulations can transition to the inactive state.


<div style="height:24px;"></div>

```css
Glucose ---> [Subpopulation A] ---> Acetate
Acetate ---> [Subpopulation B] ---> CO2

[Subpopulation A] ---> [Inactive]
[Subpopulation B] ---> [Inactive]
```

<div style="height:24px;"></div>


#### **Metabolites & Metabolome**

First, we create three `Metabolite` objects—glucose, acetate, and CO2—and group them in a `Metabolome`.


In [4]:
#import packages for the model and visualization
from kinetic_model import Metabolite, Metabolome
from kinetic_model.visualize import GraphSpecBuilder, CytoscapeExporter
import json
#to make sure plotly renders in the notebook
import plotly.io as pio
pio.renderers.default = "colab"
# Display Cytoscape visualization
from ipycytoscape import CytoscapeWidget
import ipywidgets as widgets



#create metabolites
glucose = Metabolite(name = "glucose", concentration = 5.0, formula ={'C': 6, 'H': 12, 'O': 6}, color = '#ff0000')
acetate = Metabolite(name = "acetate", concentration = 0.0, formula ={'C': 2, 'H': 4, 'O': 3}, color = '#003eff')
co2 = Metabolite(name = "co2", concentration = 0.0, formula ={'C': 1, 'O': 2}, color = '#00B8FF')

In [5]:
#create metabolome
metabolome = Metabolome(metabolites = [glucose, acetate, co2])

#visualize it
metabolome.make_plot()

We can also visualize the model components in Cytoscape

In [ ]:

#cycle through a kson file
model = json.loads(metabolome.to_json(full_model=True))

# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(model)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])
viewer.set_layout(name="preset")

# Display
display(viewer)